In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import math
import pycaw
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:

class handDetector():
    def __init__(self,mode = False,maxHands = 2,DetectionCOn = 0.5, trackcon = 0.5):
        #initialising variable for Hands thought not used here 
        self.mode = mode
        self.maxHands = maxHands
        self.DetectionCOn = DetectionCOn
        self.trackcon = trackcon
        
        #using hands funtion to get the hand detected
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands()
        self.mpDraw = mp.solutions.drawing_utils

    #drawing basic hand  
    def findHands(self, img,draw = True):
        self.imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(self.imgRGB)
        #print(results.multi_hand_landmarks)   #will print co-ordinatates of hand


        #Getting the image and masking the nodes on it
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS)
        return img


    #Add this as well below for loop if you want edge by edge
    def findPosition(self, img, handNo = 0, draw = True):
        lmlist= []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id,lm in enumerate(myHand.landmark):
                            h,w,c = img.shape
                            cx,cy = int(lm.x*w) , int(lm.y*h)
                            lmlist.append([id,cx,cy])  #this will give the point and the coordinate of the point
                            if  draw:   #Checking which id is where and creating a circle around it
                                cv2.circle(img,(cx,cy),15,(255,0,0),cv2.FILLED)    
        return lmlist





In [3]:

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)

volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(-20.0, None)
minVol = volRange[0]
maxVol = volRange[1]
print(minVol,maxVol)

-96.0 0.0


In [4]:
cap = cv2.VideoCapture(0)
detector = handDetector()
pTime = 0
vol = 0
volbar = 400
volper = 0
while True:
    success , img = cap.read()
    detector.findHands(img)
    lmlist =  detector.findPosition(img, draw = False)
    if len(lmlist) !=0:
        
        #circling the index and thumb as well as drawing a line between them
        x1,y1 = lmlist[4][1] ,lmlist[4][2]
        x2,y2 = lmlist[8][1] ,lmlist[8][2]
        cx,cy =  (x1 + x2) //2 ,(y1 +y2) //2
        cv2.circle(img,(x1,y1),5,(255,0,255),cv2.FILLED)
        cv2.circle(img,(x2,y2),5,(255,0,255),cv2.FILLED)
        cv2.line(img,(x1,y1),(x2,y2),(255,0,255),3)
        cv2.circle(img,(cx,cy),5,(255,100,0),cv2.FILLED)
        
        lenght = math.hypot(x2-x1, y2-y1)
        
        
        #Volume Changer code
        vol = np.interp(lenght,[30,200],[minVol,maxVol])
        volbar = np.interp(lenght,[30,200],[400,150])
        volper = np.interp(lenght,[30,200],[0,100])
        
        
        print(lenght)
        volume.SetMasterVolumeLevel(vol, None)
        if lenght<50:
            cv2.circle(img,(cx,cy),5,(0,255,0),cv2.FILLED)
            
            
            
    cv2.rectangle(img,(50,150),(85,400),(0,255,0),3)
    cv2.rectangle(img,(50,int(volbar)),(85,400),(0,255,0),cv2.FILLED)
    cv2.putText(img,str(int(volper)),(40,450),cv2.FONT_HERSHEY_COMPLEX,2,(250,30,32),1)        
            
            
    #fps module
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_COMPLEX,2,(250,30,32),1)
    cv2.imshow("image",img) 
    
    
    if cv2.waitKey(1) == ord('q'):
        break 

152.08221460775746
213.85976713725282
214.85343841791317
193.00259065618783
167.0269439342048
198.4363877921587
208.58571379651102
158.59697348940804
141.06027080648894
119.64113005150027
103.16006979447037
90.80198235721508
87.32124598286491
85.90692637965812
80.049984384758
75.43208866258443
71.56814934033156
66.48308055437865
61.84658438426491
46.486557196677836
38.600518131237564
40.496913462633174
42.720018726587654
39.0
36.76955262170047
37.69615364994153
35.11409973215888
34.92849839314596
33.83784863137726
35.17101079013795
36.05551275463989
34.785054261852174
30.083217912982647
26.92582403567252
29.427877939124322
31.016124838541646
24.515301344262525
23.021728866442675
26.30589287593181
36.40054944640259
40.80441152620633
48.38388161361178
54.08326913195984
96.46242791885346
105.94810050208545
112.92918134831227
130.64838307457157
136.31214179228496
137.56816492197603
160.0312469488381
163.52675622050356
166.02710622064097
178.84350701101786
185.16209115258988
190.70920271449